In [ ]:
"""Plots gridsearch figures.
"""

In [ ]:
import numpy as np

import os



In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np

In [ ]:
import hyper_pixelNN as hsinn

In [ ]:
%matplotlib ipympl

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import ipympl

In [ ]:
def str_to_spatial_spectral_receptive_field_size(s):
    """
    Coming in in (b,h,w)
    """
    filters = s.split('_')
    filters = [f.split('-') for f in filters]
    filters = [[int(n) for n in f] for f in filters]
    psi1, psi2, phi = filters
    b1,h1,_ = psi1
    b2,h2,_ = psi2
    b3,h3,_ = phi
    # return (h1+h2+h3+2, b1+b2+b3+2)
    # multiply by 3 in spectral bc there is downsampling there
    return (h1+h2+h3-2, b1*3+b2*3+b3*3-2)

In [ ]:
def results_to_grid(result_dict):

    winds = set()
    bands = set()
    best_acc = 0
    for k, v in result_dict.iteritems():
        w, b = str_to_spatial_spectral_receptive_field_size(k)
        winds.add(w)
        bands.add(b)
        best_acc = max(best_acc, sum(v) / float(len(v)))
    winds = sorted(list(winds))
    bands = sorted(list(bands))
    Z = np.zeros((len(winds), len(bands)))
#     Z[:,:] = np.nan
    
    for k, v in result_dict.iteritems():
        w, b = str_to_spatial_spectral_receptive_field_size(k)
        i = winds.index(w)
        j = bands.index(b)
        acc = sum(v) / float(len(v))
        if acc == best_acc:
            print('Best acc (%.1f) is for: %s' % (best_acc * 100, k) )
        if Z[i,j] != 0:
            print('Conflict: %s' % k)
        if Z[i,j] < acc:
            Z[i,j] = acc
        
    X, Y = np.meshgrid(np.array(bands),winds)
    
    return X, Y, Z

In [ ]:
def tap2wavelength(M, dataset):
    """Converts tap lengths to nm for a dataset's spectral axis.
    """
    return M*hsinn.bandwidth_dict[dataset]

def tap2meters(M, dataset):
    """Converts tap lengths to meters for a dataset's spatial axis.
    """
    return M*hsinn.spatial_res_dict[dataset] 
    

In [ ]:

npz_name = '/scratch0/ilya/locDoc/pyfst/gridsearch_KSC_strictsinglesite_s50_10trials.npz'
# npz_name = '/scratch0/ilya/locDoc/pyfst/gridsearch_IP_strictsinglesite_s20_10trials.npz'
# npz_name = '/scratch0/ilya/locDoc/pyfst/gridsearch_Bots_singlesite_s20_10trials.npz'
#npz_name = '/scratch0/ilya/locDoc/pyfst/gridsearch_paviaU_strictsingle_site_s20_10trials.npz'
# npz_name = '/scratch0/ilya/locDoc/pyfst/gridsearch_paviaU_strictsingle_site_s90_10trials.npz'
#npz_name = '/scratch0/ilya/locDoc/pyfst/gridsearch_paviaU_strictsingle_site_s50_10trials.npz'

In [ ]:
results = np.load(npz_name)['results'].item()

In [ ]:
X, Y, Z = results_to_grid(results)
dataset = 'KSC'
# dataset = 'Botswana'

In [ ]:
chosen_result = '5-9-9_5-5-5_5-5-5'
#chosen_result = '7-7-7_7-7-7_7-7-7'
# chosen_result = '9-7-7_9-3-3_9-3-3'
#chosen_result = '9-9-9_9-9-9_9-9-9'
print(np.array(results[chosen_result]).std())
np.argmin( np.abs((np.array(results[chosen_result]) - np.array(results[chosen_result]).mean())) )

In [ ]:
X = tap2wavelength(X, dataset)
Y = tap2meters(Y, dataset)

In [ ]:
bi, bj = [Z.argmax() // Z.shape[1], Z.argmax() % Z.shape[1]]
print('Best receptive field of %.2f m by %.2f nm' % (Y[bi,bj], X[bi,bj]))

In [ ]:
plt.close('all')

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')

real_Z = [z for z in Z.flatten() if not np.isnan(z)]

# Plot the surface.
# surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm, linewidth=0, antialiased=False, vmin=min(real_Z), vmax=max(real_Z))

# Plot the wireframe
norm = plt.Normalize(Z.min(), Z.max())
colors = cm.coolwarm(norm(Z))
rcount, ccount, _ = colors.shape
surf = ax.plot_surface(X, Y, Z, rcount=rcount, ccount=ccount,
                       facecolors=colors, shade=False, linewidth=4)
surf.set_facecolor((0,0,0,0))

# Customize the z axis.

ax.set_zlim(min(real_Z), max(real_Z))
ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))
ax.set_xlabel('Spectral Receptive Field (nm)')
ax.set_ylabel('Spatial Receptive Field (m)')
ax.set_zlabel('Accuracy')

# Add a color bar which maps values to colors.
#fig.colorbar(surf, shrink=0.5, aspect=5)